In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense

In [2]:
def incrs_order(y_pred):
    sm = 0
    li = []
    for i in range(len(y_pred)):
        for j in range(5):
            if j<=3:
                if y_pred[i][j] >= y_pred[i][j+1]:
                    print(y_pred[i])
                    li.append(i)
                    sm += 1
    return sm, li

In [3]:
stopwords = [',', 'and', 'of', 'in', 'moving']

In [4]:
df_train = pd.read_csv('train.csv')
submn = pd.read_csv('submission.csv')

In [5]:
list(df_train['PRIM_STATE'].unique())

['US',
 'RI',
 'NJ',
 'NH',
 'DE',
 'WA',
 'MI',
 'ID',
 'VA',
 'NY',
 'AR',
 'AZ',
 'TX',
 'IL',
 'MO',
 'OH',
 'UT',
 'OK',
 'WV',
 'PA',
 'IN',
 'HI',
 'KY',
 'LA',
 'MS',
 'MN',
 'NC',
 'TN',
 'ME',
 'FL',
 'NM',
 'KS',
 'CO',
 'DC',
 'MD',
 'VT',
 'MA',
 'OR',
 'CA',
 'IA',
 'AL',
 'WI',
 'CT',
 'NV',
 'SC',
 'GA',
 'AK',
 'WY',
 'NE',
 'SD',
 'MT',
 'ND']

In [6]:
list(df_train['NAICS_TITLE'].unique())

['Arts, Entertainment, and Recreation',
 'Mining, Quarrying, and Oil and Gas Extraction',
 'Finance and Insurance',
 'Administrative and Support and Waste Management and Remediation Services',
 'Manufacturing',
 'Other Services (except Public Administration)',
 'Health Care and Social Assistance',
 'Cross-industry',
 'Retail Trade',
 'Professional, Scientific, and Technical Services',
 'Information',
 'Real Estate and Rental and Leasing',
 'Wholesale Trade',
 'Management of Companies and Enterprises',
 'Federal, State, and Local Government, excluding state and local schools and hospitals and the U.S. Postal Service (OES Designation)',
 'Educational Services',
 'Utilities',
 'Agriculture, Forestry, Fishing and Hunting',
 'Construction',
 'Transportation and Warehousing',
 'Accommodation and Food Services']

In [7]:
list(df_train['OCC_TITLE'].unique())

['Supervisors of Transportation and Material Moving Workers',
 'Sales Representatives, Wholesale and Manufacturing',
 'Physical Scientists',
 'Architects, Surveyors, and Cartographers',
 'Supervisors of Protective Service Workers',
 'Supervisors of Construction and Extraction Workers',
 'Life Scientists',
 'Community and Social Service Occupations',
 'Extraction Workers',
 'Entertainment Attendants and Related Workers',
 'Counselors, Social Workers, and Other Community and Social Service Specialists',
 'Home Health and Personal Care Aides; and Nursing Assistants, Orderlies, and Psychiatric Aides',
 'Office and Administrative Support Occupations',
 'Computer and Mathematical Occupations',
 'Other Installation, Maintenance, and Repair Occupations',
 'Food Preparation and Serving Related Occupations',
 'Printing Workers',
 'Architecture and Engineering Occupations',
 'Art and Design Workers',
 'Personal Care and Service Occupations',
 'Farming, Fishing, and Forestry Occupations',
 'Transp

In [8]:
df_train[df_train['A_PCT10'] == 24660.0]

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
55,US,Manufacturing,Grounds Maintenance Workers,24660.0,29750.0,36360.0,43880.0,50900.0
59,US,Finance and Insurance,Communications Equipment Operators,24660.0,29450.0,36980.0,46500.0,53890.0
1154,NH,Cross-industry,Building and Grounds Cleaning and Maintenance ...,24660.0,29260.0,35380.0,39120.0,48470.0


In [9]:
df_train.shape

(2297, 8)

In [10]:
df_train.isnull().sum()

PRIM_STATE       0
NAICS_TITLE      0
OCC_TITLE        0
A_PCT10          0
A_PCT25          0
A_MEDIAN         0
A_PCT75         30
A_PCT90        139
dtype: int64

In [11]:
df_train[df_train.isnull().any(axis = 1)]

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
2158,US,Cross-industry,Management Occupations,47860.0,74710.0,102450.0,160960.0,NaN
2159,US,Cross-industry,Top Executives,41260.0,60900.0,98720.0,160540.0,NaN
2160,US,Cross-industry,"Advertising, Marketing, Promotions, Public Rel...",63270.0,95140.0,128160.0,177360.0,NaN
2161,US,Cross-industry,Operations Specialties Managers,73230.0,95660.0,127140.0,165860.0,NaN
2162,US,Cross-industry,Legal Occupations,38620.0,58400.0,82430.0,156900.0,NaN
...,...,...,...,...,...,...,...,...
2292,NJ,Cross-industry,Legal Occupations,47950.0,73720.0,101550.0,163990.0,NaN
2293,NY,Cross-industry,Legal Occupations,47580.0,75920.0,126220.0,NaN,NaN
2294,RI,Cross-industry,Legal Occupations,46400.0,60560.0,98070.0,161120.0,NaN
2295,TX,Cross-industry,Legal Occupations,37540.0,51260.0,80850.0,135490.0,NaN


In [12]:
# df_train.dropna(inplace = True)
# df_train.reset_index(drop = True, inplace = True)

In [13]:
sm, li = incrs_order(np.array(df_train[['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90']]))

[ 46760.  46760.  56480.  73530. 125390.]
[26000. 26000. 26000. 26000. 37590.]
[26000. 26000. 26000. 26000. 37590.]
[26000. 26000. 26000. 26000. 37590.]
[28950. 28950. 30130. 36540. 47080.]
[28400. 34910. 38790. 38790. 38790.]
[28400. 34910. 38790. 38790. 38790.]
[17290. 17290. 23520. 38290. 49810.]
[ 30690.  37220.  37220.  74620. 202280.]
[27270. 27270. 28350. 29020. 35840.]
[26000. 26000. 26000. 31630. 47330.]
[26000. 26000. 26000. 31630. 47330.]
[29640. 29640. 32550. 37720. 60220.]
[ 38390.  47860.  89400. 100940. 100940.]
[30050. 38150. 38150. 61820. 78630.]
[ 60490.  60490.  96790. 149970. 180750.]
[38020. 38020. 38910. 48370. 61700.]
[49190. 77480. 77480. 80560. 87190.]
[26000. 26000. 26000. 33640. 51800.]
[26000. 26000. 26000. 33640. 51800.]
[17290. 17290. 24330. 39200. 50170.]
[30840. 32040. 58210. 77250. 77250.]
[30020. 31180. 47780. 77620. 77620.]
[36060. 36290. 36570. 37020. 37020.]


In [14]:
print(sm, li)

24 [40, 177, 177, 177, 274, 329, 329, 461, 547, 639, 714, 714, 1165, 1178, 1355, 1417, 1475, 1673, 1713, 1713, 1927, 1949, 1967, 2107]


In [15]:
df_train.drop(li, axis = 0, inplace = True)
df_train.reset_index(drop = True, inplace = True)

In [16]:
submn.shape

(926, 8)

In [17]:
submn.isnull().sum()

PRIM_STATE     0
NAICS_TITLE    0
OCC_TITLE      0
A_PCT10        0
A_PCT25        0
A_MEDIAN       0
A_PCT75        0
A_PCT90        0
dtype: int64

In [18]:
df_train.shape

(2278, 8)

In [19]:
df_train

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,US,"Arts, Entertainment, and Recreation",Supervisors of Transportation and Material Mov...,32350.0,40200.0,50790.0,62560.0,78520.0
1,US,"Mining, Quarrying, and Oil and Gas Extraction","Sales Representatives, Wholesale and Manufactu...",47860.0,61600.0,87810.0,107460.0,153600.0
2,US,Finance and Insurance,Physical Scientists,59240.0,63050.0,89740.0,126320.0,149070.0
3,US,Administrative and Support and Waste Managemen...,"Architects, Surveyors, and Cartographers",37320.0,47630.0,60550.0,77450.0,98990.0
4,US,Manufacturing,Supervisors of Protective Service Workers,50130.0,63840.0,81770.0,104530.0,133180.0
...,...,...,...,...,...,...,...,...
2273,NJ,Cross-industry,Legal Occupations,47950.0,73720.0,101550.0,163990.0,NaN
2274,NY,Cross-industry,Legal Occupations,47580.0,75920.0,126220.0,NaN,NaN
2275,RI,Cross-industry,Legal Occupations,46400.0,60560.0,98070.0,161120.0,NaN
2276,TX,Cross-industry,Legal Occupations,37540.0,51260.0,80850.0,135490.0,NaN


Combine Both Train and Test Data

In [20]:
df_all = pd.concat([df_train, submn], axis = 0)

In [21]:
df_all.shape

(3204, 8)

In [22]:
df_all.head(5)

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,US,"Arts, Entertainment, and Recreation",Supervisors of Transportation and Material Mov...,32350.0,40200.0,50790.0,62560.0,78520.0
1,US,"Mining, Quarrying, and Oil and Gas Extraction","Sales Representatives, Wholesale and Manufactu...",47860.0,61600.0,87810.0,107460.0,153600.0
2,US,Finance and Insurance,Physical Scientists,59240.0,63050.0,89740.0,126320.0,149070.0
3,US,Administrative and Support and Waste Managemen...,"Architects, Surveyors, and Cartographers",37320.0,47630.0,60550.0,77450.0,98990.0
4,US,Manufacturing,Supervisors of Protective Service Workers,50130.0,63840.0,81770.0,104530.0,133180.0


In [23]:
df_all['PRIM_STATE'].value_counts()

US    2083
RI      22
ME      22
FL      22
NM      22
KS      22
CO      22
DC      22
MD      22
VT      22
MA      22
OR      22
CA      22
IA      22
AL      22
WI      22
CT      22
NV      22
SC      22
GA      22
AK      22
WY      22
NE      22
SD      22
MT      22
TN      22
NC      22
MN      22
TX      22
NJ      22
NH      22
DE      22
WA      22
MI      22
ID      22
VA      22
NY      22
AR      22
AZ      22
IL      22
MS      22
MO      22
OH      22
UT      22
WV      22
PA      22
IN      22
HI      22
KY      22
LA      22
ND      22
OK      21
Name: PRIM_STATE, dtype: int64

In [24]:
df_all['NAICS_TITLE'].value_counts()

Cross-industry                                                                                                                         1237
Other Services (except Public Administration)                                                                                           115
Federal, State, and Local Government, excluding state and local schools and hospitals and the U.S. Postal Service (OES Designation)     115
Administrative and Support and Waste Management and Remediation Services                                                                114
Health Care and Social Assistance                                                                                                       113
Educational Services                                                                                                                    113
Manufacturing                                                                                                                           110
Professional, Scient

In [25]:
df_all['OCC_TITLE'].value_counts()

Installation, Maintenance, and Repair Occupations                   72
Building and Grounds Cleaning and Maintenance Occupations           72
Construction and Extraction Occupations                             72
Transportation and Material Moving Occupations                      72
Production Occupations                                              72
                                                                    ..
Rail Transportation Workers                                         10
Postsecondary Teachers                                               9
Occupational Therapy and Physical Therapist Assistants and Aides     8
Law Enforcement Workers                                              6
Funeral Service Workers                                              5
Name: OCC_TITLE, Length: 116, dtype: int64

In [26]:
# new_data = df_all.copy()
# new_data['text'] = new_data['PRIM_STATE'] + new_data['NAICS_TITLE'] + new_data['OCC_TITLE']

In [27]:
# new_data

In [28]:
# f = new_data.groupby(by = 'PRIM_STATE')
# f

In [29]:
# f.head()

In [30]:
# new_data

In [31]:
# new_data[new_data['NAICS_TITLE'] == 'Manufacturing']
# # .groupby(by = 'PRIM_STATE')

In [32]:
# new_data

In [33]:
# new_data['text'].value_counts()

In [34]:
# df_all[df_all['NAICS_TITLE'] == df_all['OCC_TITLE']]

In [35]:
# df_train[df_train.isnull().any(axis = 1)]

In [36]:
# df_train[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE', 'A_PCT90']][df_train['OCC_TITLE'] == 'Management Occupations']

In [37]:
# ['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75']
# df_train[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90']][(df_train['OCC_TITLE'] == 'Management Occupations') & (df_train['A_PCT90'].isnull())]

In [38]:
# df_train[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE', 'A_PCT90']][df_train['OCC_TITLE'] == 'Management Occupations']['A_PCT90'].mode()

In [39]:
# df_train.groupby(['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE']).mean()

In [40]:
imp = IterativeImputer(random_state = 42)
df_train[['A_PCT75', 'A_PCT90']] = imp.fit_transform(df_train[['A_PCT75', 'A_PCT90']])

In [41]:
# df_train

In [42]:
# ['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75']
# df_train[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90']][(df_train['OCC_TITLE'] == 'Management Occupations')]

In [43]:
# df_all = pd.concat([df_train, submn], axis = 0)

In [44]:
# df_all

In [45]:
# df_train = df_all.iloc[:2139,]
# df_test = df_all.iloc[2139:,]

In [46]:
# submn.isnull().sum()

In [47]:
df_all

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,US,"Arts, Entertainment, and Recreation",Supervisors of Transportation and Material Mov...,32350.0,40200.0,50790.0,62560.0,78520.0
1,US,"Mining, Quarrying, and Oil and Gas Extraction","Sales Representatives, Wholesale and Manufactu...",47860.0,61600.0,87810.0,107460.0,153600.0
2,US,Finance and Insurance,Physical Scientists,59240.0,63050.0,89740.0,126320.0,149070.0
3,US,Administrative and Support and Waste Managemen...,"Architects, Surveyors, and Cartographers",37320.0,47630.0,60550.0,77450.0,98990.0
4,US,Manufacturing,Supervisors of Protective Service Workers,50130.0,63840.0,81770.0,104530.0,133180.0
...,...,...,...,...,...,...,...,...
921,US,Utilities,Other Educational Instruction and Library Occu...,10000.0,20000.0,30000.0,40000.0,50000.0
922,US,"Arts, Entertainment, and Recreation",Community and Social Service Occupations,10000.0,20000.0,30000.0,40000.0,50000.0
923,WI,Cross-industry,"Arts, Design, Entertainment, Sports, and Media...",10000.0,20000.0,30000.0,40000.0,50000.0
924,PA,Cross-industry,Computer and Mathematical Occupations,10000.0,20000.0,30000.0,40000.0,50000.0


In [48]:
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.target_encoder import TargetEncoder
CB_encoder = TargetEncoder()

/opt/conda/lib/python3.9/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/opt/conda/lib/python3.9/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [49]:
df_all.dtypes

PRIM_STATE      object
NAICS_TITLE     object
OCC_TITLE       object
A_PCT10        float64
A_PCT25        float64
A_MEDIAN       float64
A_PCT75        float64
A_PCT90        float64
dtype: object

In [50]:
inp = df_all.select_dtypes(exclude = 'float').columns.to_list()
out = df_all.select_dtypes(include = 'float').columns.to_list()

In [51]:
inp

['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE']

In [52]:
for i in inp:
    for j in out:
        CB_encoder.fit(df_all[i], df_all[j])
        df_all[i+'_'+j] = CB_encoder.transform(df_all[i])

In [53]:
df_all['text'] = df_all['PRIM_STATE'] + '' + df_all['NAICS_TITLE'] + '' + df_all['OCC_TITLE']

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range = (1,3), max_features = 15, stop_words = stopwords)
t = tfidf.fit_transform(df_all['text'])
df = pd.DataFrame(t.toarray(), index = range(0,3204), columns = tfidf.get_feature_names_out())

In [55]:
df_all

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,...,NAICS_TITLE_A_PCT25,NAICS_TITLE_A_MEDIAN,NAICS_TITLE_A_PCT75,NAICS_TITLE_A_PCT90,OCC_TITLE_A_PCT10,OCC_TITLE_A_PCT25,OCC_TITLE_A_MEDIAN,OCC_TITLE_A_PCT75,OCC_TITLE_A_PCT90,text
0,US,"Arts, Entertainment, and Recreation",Supervisors of Transportation and Material Mov...,32350.0,40200.0,50790.0,62560.0,78520.0,27845.223236,37339.111858,...,31228.348624,41786.146789,52725.420561,71185.420561,23768.571435,33159.047625,44226.666676,55829.523825,68981.428587,"USArts, Entertainment, and RecreationSuperviso..."
1,US,"Mining, Quarrying, and Oil and Gas Extraction","Sales Representatives, Wholesale and Manufactu...",47860.0,61600.0,87810.0,107460.0,153600.0,27845.223236,37339.111858,...,44023.513514,59762.702703,77691.506849,80067.500000,26032.000005,38774.999986,54481.499968,75350.999933,98424.499877,"USMining, Quarrying, and Oil and Gas Extractio..."
2,US,Finance and Insurance,Physical Scientists,59240.0,63050.0,89740.0,126320.0,149070.0,27845.223236,37339.111858,...,47015.384615,62525.384615,78699.418605,88653.670886,37214.116499,50338.233710,72393.526754,95454.365187,111544.206613,USFinance and InsurancePhysical Scientists
3,US,Administrative and Support and Waste Managemen...,"Architects, Surveyors, and Cartographers",37320.0,47630.0,60550.0,77450.0,98990.0,27845.223236,37339.111858,...,33274.385965,44995.263158,60683.157895,76234.954955,37038.947216,49413.684010,65139.473435,81223.683939,102987.368018,USAdministrative and Support and Waste Managem...
4,US,Manufacturing,Supervisors of Protective Service Workers,50130.0,63840.0,81770.0,104530.0,133180.0,27845.223236,37339.111858,...,40199.363636,54019.545455,66817.222222,77580.970874,34002.221933,44014.444123,58942.221801,75161.110623,94293.332599,USManufacturingSupervisors of Protective Servi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,US,Utilities,Other Educational Instruction and Library Occu...,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,...,52460.256410,68138.846154,82725.974026,90757.361111,21010.626835,28190.627469,39320.627892,52255.628402,71228.126628,USUtilitiesOther Educational Instruction and L...
922,US,"Arts, Entertainment, and Recreation",Community and Social Service Occupations,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,...,31228.348624,41786.146789,52725.420561,71185.420561,25666.764706,33725.147059,44382.647059,56891.176471,70301.029412,"USArts, Entertainment, and RecreationCommunity..."
923,WI,Cross-industry,"Arts, Design, Entertainment, Sports, and Media...",10000.0,20000.0,30000.0,40000.0,50000.0,22735.000003,31593.181822,...,34376.782538,46627.890057,62402.576985,77922.226856,22812.394366,31721.408451,44838.732394,61594.507042,83428.732394,"WICross-industryArts, Design, Entertainment, S..."
924,PA,Cross-industry,Computer and Mathematical Occupations,10000.0,20000.0,30000.0,40000.0,50000.0,21918.636367,31267.272731,...,34376.782538,46627.890057,62402.576985,77922.226856,36485.211268,50788.309859,71777.042254,94942.535211,118339.014085,PACross-industryComputer and Mathematical Occu...


In [56]:
df
# t.toarray()

,care,entertainment,local,maintenance,occupations,related,service,service occupations,social,state,state local,support,support occupations,technical,workers
0,0.0,0.860611,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.509262
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,1.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.796968,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.604021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3200,0.0,0.535128,0.0,0.0,0.225963,0.0,0.417812,0.515333,0.47164,0.0,0.0,0.0,0.0,0.0,0.000000
3201,0.0,0.921237,0.0,0.0,0.389001,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3202,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000


In [57]:
# heh
df_all.columns

Index(['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE', 'A_PCT10', 'A_PCT25',
       'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'PRIM_STATE_A_PCT10',
       'PRIM_STATE_A_PCT25', 'PRIM_STATE_A_MEDIAN', 'PRIM_STATE_A_PCT75',
       'PRIM_STATE_A_PCT90', 'NAICS_TITLE_A_PCT10', 'NAICS_TITLE_A_PCT25',
       'NAICS_TITLE_A_MEDIAN', 'NAICS_TITLE_A_PCT75', 'NAICS_TITLE_A_PCT90',
       'OCC_TITLE_A_PCT10', 'OCC_TITLE_A_PCT25', 'OCC_TITLE_A_MEDIAN',
       'OCC_TITLE_A_PCT75', 'OCC_TITLE_A_PCT90', 'text'],
      dtype='object')

In [58]:
df_all.reset_index(drop = True, inplace =True)
df.reset_index(drop = True, inplace = True)

In [59]:
df_all.drop(['text'], axis = 1, inplace = True)
df_all = pd.concat([df_all, df], axis = 1)

In [60]:
df_all
# tfidf.get_feature_names_out()

,PRIM_STATE,NAICS_TITLE,OCC_TITLE,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,...,related,service,service occupations,social,state,state local,support,support occupations,technical,workers
0,US,"Arts, Entertainment, and Recreation",Supervisors of Transportation and Material Mov...,32350.0,40200.0,50790.0,62560.0,78520.0,27845.223236,37339.111858,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.509262
1,US,"Mining, Quarrying, and Oil and Gas Extraction","Sales Representatives, Wholesale and Manufactu...",47860.0,61600.0,87810.0,107460.0,153600.0,27845.223236,37339.111858,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
2,US,Finance and Insurance,Physical Scientists,59240.0,63050.0,89740.0,126320.0,149070.0,27845.223236,37339.111858,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3,US,Administrative and Support and Waste Managemen...,"Architects, Surveyors, and Cartographers",37320.0,47630.0,60550.0,77450.0,98990.0,27845.223236,37339.111858,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,1.0,0.0,0.0,0.000000
4,US,Manufacturing,Supervisors of Protective Service Workers,50130.0,63840.0,81770.0,104530.0,133180.0,27845.223236,37339.111858,...,0.0,0.796968,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.604021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,US,Utilities,Other Educational Instruction and Library Occu...,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3200,US,"Arts, Entertainment, and Recreation",Community and Social Service Occupations,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,...,0.0,0.417812,0.515333,0.47164,0.0,0.0,0.0,0.0,0.0,0.000000
3201,WI,Cross-industry,"Arts, Design, Entertainment, Sports, and Media...",10000.0,20000.0,30000.0,40000.0,50000.0,22735.000003,31593.181822,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
3202,PA,Cross-industry,Computer and Mathematical Occupations,10000.0,20000.0,30000.0,40000.0,50000.0,21918.636367,31267.272731,...,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000


In [61]:
# heh
df_all = pd.get_dummies(df_all, columns = ['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'], drop_first = True)

In [62]:
# df_all.drop(inp, axis = 1, inplace = True)

In [63]:
df_all

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
0,32350.0,40200.0,50790.0,62560.0,78520.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,1,0,0,0,0,0,0,0
1,47860.0,61600.0,87810.0,107460.0,153600.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2,59240.0,63050.0,89740.0,126320.0,149070.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3,37320.0,47630.0,60550.0,77450.0,98990.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
4,50130.0,63840.0,81770.0,104530.0,133180.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3200,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3201,10000.0,20000.0,30000.0,40000.0,50000.0,22735.000003,31593.181822,43005.000004,58723.636367,77201.363636,...,0,0,0,0,0,0,0,0,0,0
3202,10000.0,20000.0,30000.0,40000.0,50000.0,21918.636367,31267.272731,42742.727277,59153.636367,70972.857154,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# cat_x = df_all[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE']]
# cat_y = df_all.drop(['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'], axis = 1)

# cat_x = cat_x.iloc[:2297,]
# cat_y = cat_y.iloc[:2297,]

In [65]:
# cat_x

In [66]:
# cat_y

In [67]:
# heh

In [68]:
# categ_data = CB_encoder.fit_transform(cat_x, cat_y)

In [69]:
#onehot encoding categorical 3 features
# encoder = OneHotEncoder()
# df_all[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE']] = encoder.fit_transform(df_all[['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE']])
# df_all.drop(['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'], axis = 1, inplace = True)

In [70]:
df_all.columns

Index(['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90',
       'PRIM_STATE_A_PCT10', 'PRIM_STATE_A_PCT25', 'PRIM_STATE_A_MEDIAN',
       'PRIM_STATE_A_PCT75', 'PRIM_STATE_A_PCT90',
       ...
       'OCC_TITLE_Supervisors of Protective Service Workers',
       'OCC_TITLE_Supervisors of Sales Workers',
       'OCC_TITLE_Supervisors of Transportation and Material Moving Workers',
       'OCC_TITLE_Textile, Apparel, and Furnishings Workers',
       'OCC_TITLE_Top Executives', 'OCC_TITLE_Tour and Travel Guides',
       'OCC_TITLE_Transportation and Material Moving Occupations',
       'OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers',
       'OCC_TITLE_Water Transportation Workers', 'OCC_TITLE_Woodworkers'],
      dtype='object', length=221)

In [71]:
df_all

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
0,32350.0,40200.0,50790.0,62560.0,78520.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,1,0,0,0,0,0,0,0
1,47860.0,61600.0,87810.0,107460.0,153600.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2,59240.0,63050.0,89740.0,126320.0,149070.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3,37320.0,47630.0,60550.0,77450.0,98990.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
4,50130.0,63840.0,81770.0,104530.0,133180.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3200,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3201,10000.0,20000.0,30000.0,40000.0,50000.0,22735.000003,31593.181822,43005.000004,58723.636367,77201.363636,...,0,0,0,0,0,0,0,0,0,0
3202,10000.0,20000.0,30000.0,40000.0,50000.0,21918.636367,31267.272731,42742.727277,59153.636367,70972.857154,...,0,0,0,0,0,0,0,0,0,0


In [72]:
df_train = df_all.iloc[:2139,]
df_test = df_all.iloc[2139:,]

In [73]:
df_test

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
2139,47860.0,74710.0,102450.0,160960.0,NaN,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2140,41260.0,60900.0,98720.0,160540.0,NaN,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,1,0,0,0,0,0
2141,63270.0,95140.0,128160.0,177360.0,NaN,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2142,73230.0,95660.0,127140.0,165860.0,NaN,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2143,38620.0,58400.0,82430.0,156900.0,NaN,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3200,10000.0,20000.0,30000.0,40000.0,50000.0,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3201,10000.0,20000.0,30000.0,40000.0,50000.0,22735.000003,31593.181822,43005.000004,58723.636367,77201.363636,...,0,0,0,0,0,0,0,0,0,0
3202,10000.0,20000.0,30000.0,40000.0,50000.0,21918.636367,31267.272731,42742.727277,59153.636367,70972.857154,...,0,0,0,0,0,0,0,0,0,0


In [74]:
df_test.drop(['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90'], axis = 1, inplace = True)

/tmp/ipykernel_152731/1984714997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90'], axis = 1, inplace = True)


In [75]:
df_train

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
0,32350.0,40200.0,50790.0,62560.0,78520.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,1,0,0,0,0,0,0,0
1,47860.0,61600.0,87810.0,107460.0,153600.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2,59240.0,63050.0,89740.0,126320.0,149070.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
3,37320.0,47630.0,60550.0,77450.0,98990.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
4,50130.0,63840.0,81770.0,104530.0,133180.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,37540.0,47670.0,62370.0,81660.0,99340.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2135,38200.0,48400.0,61850.0,79360.0,101260.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2136,22480.0,28690.0,36030.0,39890.0,52020.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0
2137,29580.0,36120.0,46180.0,61810.0,80320.0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,...,0,0,0,0,0,0,0,0,0,0


In [76]:
df_test

,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,NAICS_TITLE_A_PCT10,NAICS_TITLE_A_PCT25,NAICS_TITLE_A_MEDIAN,NAICS_TITLE_A_PCT75,NAICS_TITLE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
2139,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0
2140,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,1,0,0,0,0,0
2141,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0
2142,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0
2143,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,39163.205128,52460.256410,68138.846154,82725.974026,90757.361111,...,0,0,0,0,0,0,0,0,0,0
3200,27845.223236,37339.111858,49990.859337,64021.045720,75605.149367,23201.834862,31228.348624,41786.146789,52725.420561,71185.420561,...,0,0,0,0,0,0,0,0,0,0
3201,22735.000003,31593.181822,43005.000004,58723.636367,77201.363636,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0
3202,21918.636367,31267.272731,42742.727277,59153.636367,70972.857154,25570.105093,34376.782538,46627.890057,62402.576985,77922.226856,...,0,0,0,0,0,0,0,0,0,0


In [77]:
X = df_train.drop(['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90'], axis = 1)
y = df_train[['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90']]

In [78]:
y

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,32350.0,40200.0,50790.0,62560.0,78520.0
1,47860.0,61600.0,87810.0,107460.0,153600.0
2,59240.0,63050.0,89740.0,126320.0,149070.0
3,37320.0,47630.0,60550.0,77450.0,98990.0
4,50130.0,63840.0,81770.0,104530.0,133180.0
...,...,...,...,...,...
2134,37540.0,47670.0,62370.0,81660.0,99340.0
2135,38200.0,48400.0,61850.0,79360.0,101260.0
2136,22480.0,28690.0,36030.0,39890.0,52020.0
2137,29580.0,36120.0,46180.0,61810.0,80320.0


In [79]:
X

,PRIM_STATE_A_PCT10,PRIM_STATE_A_PCT25,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,NAICS_TITLE_A_PCT10,NAICS_TITLE_A_PCT25,NAICS_TITLE_A_MEDIAN,NAICS_TITLE_A_PCT75,NAICS_TITLE_A_PCT90,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
0,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,23201.834862,31228.348624,41786.146789,52725.420561,71185.420561,...,0,0,1,0,0,0,0,0,0,0
1,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,32223.108108,44023.513514,59762.702703,77691.506849,80067.500000,...,0,0,0,0,0,0,0,0,0,0
2,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,35467.912088,47015.384615,62525.384615,78699.418605,88653.670886,...,0,0,0,0,0,0,0,0,0,0
3,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,24873.421053,33274.385965,44995.263158,60683.157895,76234.954955,...,0,0,0,0,0,0,0,0,0,0
4,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,29977.363636,40199.363636,54019.545455,66817.222222,77580.970874,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,28906.347826,38774.086957,51906.434783,67473.391304,81238.596491,...,0,0,0,0,0,0,0,0,0,0
2135,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,24742.920354,33541.769912,44270.707965,57066.371681,67298.909091,...,0,0,0,0,0,0,0,0,0,0
2136,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,24742.920354,33541.769912,44270.707965,57066.371681,67298.909091,...,0,0,0,0,0,0,0,0,0,0
2137,27845.223236,37339.111858,49990.859337,64021.04572,75605.149367,25828.938053,34588.407080,46223.274336,57637.477477,68712.222222,...,0,0,0,0,0,0,0,0,0,0


In [80]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X)
# Y = y
# Y = scaler.fit_transform(y)

In [81]:
# x

In [82]:
# Y

Feature Selection

In [83]:
from sklearn.feature_selection import RFECV

In [84]:
from sklearn.linear_model import Lasso

In [85]:
estimator = Lasso(random_state = 42)
selector = RFECV(estimator, cv = 5, verbose = True)
selector = selector.fit(x, y)
selector.support_

Fitting estimator with 216 features.


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.599e+08, tolerance: 1.927e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+08, tolerance: 3.773e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.371e+09, tolerance: 7.766e+07

Fitting estimator with 215 features.


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.599e+08, tolerance: 1.927e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+08, tolerance: 3.773e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.371e+09, tolerance: 7.766e+07

Fitting estimator with 214 features.


...

Fitting estimator with 203 features.


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.497e+08, tolerance: 2.405e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.684e+08, tolerance: 4.707e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.865e+08, tolerance: 9.581e+07

Fitting estimator with 202 features.


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.334e+08, tolerance: 2.405e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.682e+08, tolerance: 4.707e+07
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.860e+08, tolerance: 9.581e+07

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,

In [86]:
fs = X.columns[selector.support_].to_list()

In [87]:
fs

['PRIM_STATE_A_PCT10',
 'PRIM_STATE_A_MEDIAN',
 'PRIM_STATE_A_PCT75',
 'PRIM_STATE_A_PCT90',
 'NAICS_TITLE_A_PCT10',
 'NAICS_TITLE_A_PCT25',
 'NAICS_TITLE_A_MEDIAN',
 'NAICS_TITLE_A_PCT75',
 'NAICS_TITLE_A_PCT90',
 'OCC_TITLE_A_PCT10',
 'OCC_TITLE_A_PCT25',
 'OCC_TITLE_A_MEDIAN',
 'OCC_TITLE_A_PCT75',
 'OCC_TITLE_A_PCT90',
 'care',
 'entertainment',
 'local',
 'occupations',
 'service',
 'service occupations',
 'social',
 'state',
 'support',
 'support occupations',
 'technical',
 'workers',
 'PRIM_STATE_AL',
 'PRIM_STATE_AR',
 'PRIM_STATE_AZ',
 'PRIM_STATE_CA',
 'PRIM_STATE_CO',
 'PRIM_STATE_CT',
 'PRIM_STATE_DC',
 'PRIM_STATE_DE',
 'PRIM_STATE_FL',
 'PRIM_STATE_GA',
 'PRIM_STATE_IA',
 'PRIM_STATE_ID',
 'PRIM_STATE_IL',
 'PRIM_STATE_IN',
 'PRIM_STATE_KS',
 'PRIM_STATE_KY',
 'PRIM_STATE_LA',
 'PRIM_STATE_MA',
 'PRIM_STATE_ME',
 'PRIM_STATE_MI',
 'PRIM_STATE_MN',
 'PRIM_STATE_MO',
 'PRIM_STATE_MS',
 'PRIM_STATE_MT',
 'PRIM_STATE_NC',
 'PRIM_STATE_ND',
 'PRIM_STATE_NE',
 'PRIM_STATE_NH',

In [88]:
# fs = 
selector.get_feature_names_out()

array(['x0', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
       'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x19', 'x21', 'x22',
       'x23', 'x24', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32',
       'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x41', 'x42',
       'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x50', 'x51', 'x52',
       'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61',
       'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70',
       'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x80',
       'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x88', 'x89', 'x91',
       'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100',
       'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108',
       'x109', 'x111', 'x112', 'x113', 'x115', 'x116', 'x117', 'x118',
       'x119', 'x121', 'x122', 'x123', 'x124', 'x125', 'x126', 'x127',
       'x128', 'x129', 'x130', 'x131', 'x132', 'x133', 'x134', 'x135',
       'x1

In [89]:
# X = pd.DataFrame(x, columns = X.columns)
# y = pd.DataFrame(Y, columns = y.columns)
X = pd.DataFrame(x, columns = X.columns)
y = pd.DataFrame(y, columns = y.columns)
X = X[fs]
df_test = df_test[fs]

In [90]:
X

,PRIM_STATE_A_PCT10,PRIM_STATE_A_MEDIAN,PRIM_STATE_A_PCT75,PRIM_STATE_A_PCT90,NAICS_TITLE_A_PCT10,NAICS_TITLE_A_PCT25,NAICS_TITLE_A_MEDIAN,NAICS_TITLE_A_PCT75,NAICS_TITLE_A_PCT90,OCC_TITLE_A_PCT10,...,OCC_TITLE_Supervisors of Protective Service Workers,OCC_TITLE_Supervisors of Sales Workers,OCC_TITLE_Supervisors of Transportation and Material Moving Workers,"OCC_TITLE_Textile, Apparel, and Furnishings Workers",OCC_TITLE_Top Executives,OCC_TITLE_Tour and Travel Guides,OCC_TITLE_Transportation and Material Moving Occupations,"OCC_TITLE_Vehicle and Mobile Equipment Mechanics, Installers, and Repairers",OCC_TITLE_Water Transportation Workers,OCC_TITLE_Woodworkers
0,0.418129,0.407463,0.412164,0.291002,0.047330,0.058893,0.067886,0.032875,0.309788,0.138365,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.418129,0.407463,0.412164,0.291002,0.585773,0.626040,0.703730,0.837704,0.623018,0.177500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.418129,0.407463,0.412164,0.291002,0.779443,0.758655,0.801448,0.870196,0.925813,0.370844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.418129,0.407463,0.412164,0.291002,0.147100,0.149584,0.181395,0.289408,0.487861,0.367816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.418129,0.407463,0.412164,0.291002,0.451734,0.456534,0.500590,0.487151,0.535329,0.315309,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,0.418129,0.407463,0.412164,0.291002,0.387809,0.393359,0.425848,0.508304,0.664317,0.174135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2135,0.418129,0.407463,0.412164,0.291002,0.139311,0.161436,0.155767,0.172814,0.172728,0.313871,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2136,0.418129,0.407463,0.412164,0.291002,0.139311,0.161436,0.155767,0.172814,0.172728,0.094073,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2137,0.418129,0.407463,0.412164,0.291002,0.204131,0.207828,0.224830,0.191225,0.222569,0.306642,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
y

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,32350.0,40200.0,50790.0,62560.0,78520.0
1,47860.0,61600.0,87810.0,107460.0,153600.0
2,59240.0,63050.0,89740.0,126320.0,149070.0
3,37320.0,47630.0,60550.0,77450.0,98990.0
4,50130.0,63840.0,81770.0,104530.0,133180.0
...,...,...,...,...,...
2134,37540.0,47670.0,62370.0,81660.0,99340.0
2135,38200.0,48400.0,61850.0,79360.0,101260.0
2136,22480.0,28690.0,36030.0,39890.0,52020.0
2137,29580.0,36120.0,46180.0,61810.0,80320.0


In [92]:
# x

In [93]:
# y

In [94]:
# x = df_train.drop(['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90'], axis = 1)
# Y = df_train[['A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90']]

In [95]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import HistGradientBoostingRegressor

/opt/conda/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [96]:
def mnwae(y_true, y_pred):
    abs_pred = np.abs((y_pred-y_true))
    # print(abs_pred)
    weigh = [0.1, 0.2, 0.4, 0.2, 0.1]
    sm = 0
    for i in range(len(abs_pred)):
        for j in range(len(weigh)):
            if (y_true[i][j] > 0) or (y_true[i][j] < 0):
                sm += ((weigh[j] * abs_pred[i][j])/y_true[i][j])
    # print(len(abs_pred))
    sm = sm/len(abs_pred)
    # print(sm)
    return sm

In [97]:
# def check_pred(y_true, y_pred):
#     sm = 0
#     diff_pred = (y_pred - y_true)
#     for i in range(len(diff_pred)):
#         for j in range(5):
#             if diff_pred[i][j] < 0:
#                 sm += 1
#     print(sm)

In [98]:
# def incrs_order(y_pred):
#     sm = 0
#     li = []
#     for i in range(len(y_pred)):
#         for j in range(5):
#             if j<=3:
#                 if y_pred[i][j] >= y_pred[i][j+1]:
#                     print(y_pred[i])
#                     li.append(i)
#                     sm += 1
#     return sm, li

In [99]:
# y_true = [[10000, 30000, 60000, 80000, 100000]]
# y_pred = [[11000, 33000, 54000, 8000, 120000]]
# print(mnwae(np.array(y_true), np.array(y_pred)))

In [100]:
# check_pred(np.array(y_true), np.array(y_pred))

In [101]:
# np.array(Y)

In [102]:
# sm, li = incrs_order(np.array(y))

In [103]:
# print(sm)

In [104]:
# print(li)

In [105]:
# y.iloc[li]

In [106]:
# y

In [107]:
# X

In [108]:
# y

In [109]:
# X

In [110]:
# X.drop([83, 366, 164, 359, 430], axis = 0, inplace = True)
# y.drop([83, 366, 164, 359, 430], axis = 0, inplace = True)
# X.reset_index(drop = True, inplace = True)
# y.reset_index(drop = True, inplace = True)

In [111]:
y

,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90
0,32350.0,40200.0,50790.0,62560.0,78520.0
1,47860.0,61600.0,87810.0,107460.0,153600.0
2,59240.0,63050.0,89740.0,126320.0,149070.0
3,37320.0,47630.0,60550.0,77450.0,98990.0
4,50130.0,63840.0,81770.0,104530.0,133180.0
...,...,...,...,...,...
2134,37540.0,47670.0,62370.0,81660.0,99340.0
2135,38200.0,48400.0,61850.0,79360.0,101260.0
2136,22480.0,28690.0,36030.0,39890.0,52020.0
2137,29580.0,36120.0,46180.0,61810.0,80320.0


In [112]:
# X['PRIM_STATE'] = X['PRIM_STATE'].astype('str')

In [113]:
# x.dtypes

In [114]:
# for c in x.columns:
#     col_type = x[c].dtype
#     if col_type == 'object' or col_type.name == 'category':
#         x[c] = x[c].astype('category')

In [115]:
# x.dtypes

In [116]:
# x
# X.dtypes

In [117]:
# Y
# df_test = df_test.astype('str')

In [118]:
# xtr.dtypes

In [119]:
skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
i = 1
pred_val_full = []
cv_mnwae = []
for train_index, test_index in skf.split(X, y):
    print('{} of KFold {}'.format(i, skf.n_splits))
    
    xtr, xval = X.iloc[train_index], X.iloc[test_index]
    ytr, yval = y.iloc[train_index], y.iloc[test_index]
    
    # p_tr = Pool(xtr, ytr, cat_features = ['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'])
    # p_val = Pool(xval, cat_features = ['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'])
    # p_te = Pool(df_test, cat_features = ['PRIM_STATE', 'NAICS_TITLE', 'OCC_TITLE'])
    
    model = RandomForestRegressor(random_state = 42, n_jobs = -1)
    # model = KNeighborsRegressor()
    # model = DecisionTreeRegressor(random_state = 42)
    # model = LinearRegression()
    # model = ExtraTreeRegressor(random_state = 42)
    # model = LGBMRegressor(random_state = 42, objective = 'regression')
    # model = xgb.XGBRegressor(random_state = 42)
    # model = LinearSVR()
    # model = HuberRegressor()
    # model = RANSACRegressor()
    # model = TheilSenRegressor()
    # model = Lasso(random_state = 42)
    # model = Ridge(random_state = 42)
    # model = RidgeCV()
    # model = ElasticNet(random_state = 42)
    # model = GradientBoostingRegressor(random_state = 42)
    # model = AdaBoostRegressor(random_state = 42)
    # model = CatBoostRegressor(random_state = 42, verbose = False)
    # model = HistGradientBoostingRegressor(random_state = 42)
    model.fit(xtr, ytr)
    # wrapper = RegressorChain(model)
    # wrapper.fit(xtr,ytr)
    # wrapper.fit(p_tr)
    # xval
    y_pred = model.predict(xval)
    test_pred = model.predict(df_test)
    # test_pred = wrapper.predict(df_test)
    # test_pred = wrapper.predict(p_te)
    # y_pred = wrapper.predict(xval)
    # y_pred = wrapper.predict(p_val)
    print("Train non increasing order = ", incrs_order(np.array(y_pred)))
    print("-----------")
    print("TEST non increasing order = ", incrs_order(np.array(test_pred)))

    # check_pred(np.array(yval), np.array(y_pred))
    score = mnwae(np.array(yval), np.array(y_pred))
    print("Validation MNWAE = {}".format(score))
    cv_mnwae.append(score)
    pred_val_full.append(y_pred)
    i += 1

1 of KFold 5
Train non increasing order =  (0, [])
-----------
TEST non increasing order =  (0, [])
Validation MNWAE = 0.10751937714723067
2 of KFold 5
Train non increasing order =  (0, [])
-----------
TEST non increasing order =  (0, [])
Validation MNWAE = 0.10622278089684502
3 of KFold 5
Train non increasing order =  (0, [])
-----------
TEST non increasing order =  (0, [])
Validation MNWAE = 0.11452947468690472
4 of KFold 5
Train non increasing order =  (0, [])
-----------
TEST non increasing order =  (0, [])
Validation MNWAE = 0.10369556924194731
5 of KFold 5
Train non increasing order =  (0, [])
-----------
TEST non increasing order =  (0, [])
Validation MNWAE = 0.09901742202241956


In [120]:
print(np.mean(cv_mnwae))

0.10619692479906946


rf - 0.1059094 10 max tfidf features + ohe features + target encoding features + rfecv fs
rf - 0.1061165 ohe + te + rfecv fs

Dropped rows with non increasing order(should have filled with nans)
imputed nan with iterative imputer
tfidf features + ohe+ targetencoding features
rf - 0.10733

#regchain
histgrad - 0.1106
lgbm - 0.11077
rf_regreorchain - 0.113666
huberrgchain - 0.15404
knn - 0.16934
gb - 0.1184
ridgecv  0.1224
adaboost - 0.2722
elastic - 0.3089
linearsvc - 0.88580

tf-idf 100 features + te + feature selection
#regression chain
catboost - 0.1090300
histgboost - 0.111
rf - 0.1149
knn - 0.1634
lgbm - 0.1123
huber - 0.1646
ridge - 0.13216
aboost - 0.26699
enet - 0.30982


TF-idf with 100max features + feature selection
normal models
RF - 0.26
knn - 0.299
dtree - 0.269
extratree - 0.2706
elasticnet - 0.354

regressorchain
huber - 0.275
catb - 0.2738
rf - 0.2857
adaboost - 0.4447
histgrad - 0.277
few other also working lazy


rf -0.1067 for tfidf + ohe + te +fs

Just models(Multioutput enabled) TE with iterative imputer
RF - 0.10727
knn - 0.11557
dtree - 0.140255
extratree - 0.14476

REGRESSOR CHAIN TE with Iterative imputer
RF - 0.1154
huber - 0.1418
adaboost - 0.264728

REGRESSOR CHAIN OHE with iterative imputer
catboost - 0.14867
HuberRegressor - 0.158405
RandomForest - 0.212436
lightgbm - 0.2477
histgradientboostregressor - 0.24907
KNN - 0.28548
linearsvr - 0.90915
elasticnet - 0.3541
gb = 0.24774
adaboost - 0.54821

In [121]:
# x.iloc[[83, 366, 164, 359, 430],]

In [122]:
# Y.iloc[[83, 366, 164, 359, 430],]

#normal models
lr - 0.143206
lasso - 0.1433
ridgecv - 0.1442
ridge - 0.1452
knn - 0.3228
rf - 0.3399
elasticnet - 0.4027
extratree - 0.4431
dtree - 0.4438
ransac - 7698

#regressorChain
lr - 0.1432
lasso - 0.1436
ridgecv - 0.1440
ridge - 0.1452
xgb - 0.1626
huberregressor - 0.17131
lgbm - 0.2806
gb - 0.2828
theilsen regressor - 0.30141
knn = 0.35589
rf - 0.3930
dtree - 0.573029
extratree - 0.5740
adaboost - 0.6132
linearsvc - 0.8977
ransac regressor - 71952

In [123]:
import keras as K
import tensorflow as tf

In [124]:
def mnwae(y_true, y_pred):
    # y_true = np.array(y_true)
    # y_true.numpy()
    # y_pred.numpy()
    # y_pred = np.array(y_pred)
    # tf.cast(y_true, tf.float32)
    # tf.cast(y_pred, tf.float32)
    abs_pred = K.backend.abs((y_pred-y_true))
    # print(abs_pred)
    # tf.cast(abs_pred, tf.float32)
    # perc = [10, 25, 50, 75, 90]
    weigh = [0.1, 0.2, 0.4, 0.2, 0.1]
    # tf.cast(weigh, tf.float32)
    sm = 0
    # tf.cast(sm, tf.float32)
    for i in range(len(abs_pred)):
        for j in range(len(weigh)):
            sm += ((weigh[j] * abs_pred.numpy()[i][j])/y_true.numpy()[i][j])
    sm = sm/len(abs_pred)
    print(sm)
    return sm

In [125]:
y_true = [[10000, 30000, 60000, 80000, 100000]]
y_pred = [[11000, 33000, 54000, 88000, 120000]]
print(mnwae(np.array(y_true), np.array(y_pred)))

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
Y

In [ ]:
# # y_bins = pd.qcut(y, q = 5, labels = [0,1,2,3,4])
# import keras
# n_classes = len(np.unique(y))
# y_bins = keras.utils.to_categorical(y, n_classes)

In [ ]:
# y_max = y.argmax(axis = 1)

In [ ]:
# y_max

In [ ]:
skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
i = 1
pred_val_full = []
cv_mnwae = []
for train_index, test_index in skf.split(x, Y):
    print('{} of KFold {}'.format(i, skf.n_splits))
    
    xtr, xval = x.iloc[train_index], x.iloc[test_index]
    ytr, yval = Y.iloc[train_index], Y.iloc[test_index]
    # tf.cast(xtr, tf.float32)
    # tf.cast(xval, tf.float32)
    model = Sequential([
        Dense(186, activation = 'relu', input_shape=(186,)),
        Dense(744, activation = 'relu'),
        Dense(372, activation = 'relu'),
        Dense(5, activation = 'sigmoid')
    ])
    # print(type(xtr))
    model.compile(optimizer = 'adam', loss = mnwae, metrics = ['mean_normal_weighted_average_error'], run_eagerly = True)
    model.fit(xtr, ytr, epochs = 5, batch_size = 25000)
    y_pred = model.predict(xval)
    score = mnwae(yval, y_pred)
    print("Validation MNWAE = {}".format(score))
    cv_mnwae.append(score)
    pred_val_full.append(y_pred)
    i += 1

In [ ]:
print(np.mean(cv_mnwae))